# 5.6 AlexNet

计算机视觉流程中真正重要的是数据和特征。也就是说，使用较干净的数据集和较有效的特征甚至比机器学习模型的选择对图像分类结果的影响更大。



In [1]:
import torch
from torch import nn,optim
import torchvision
import d2l_pytorch as d2l 
import time
sys.path.append('..')

In [3]:
num_classes=10
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet,self).__init__()
        self.conv=nn.Sequential(
            # layer 1
            nn.Conv2d(in_channels=3,out_channels=96,kernel_size=11,stride=4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3,stride=2),
            # layer 2
            nn.Conv2d(in_channels=96,out_channels=256,kernel_size=5,groups=2,padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3,stride=2),
            # layer 3
            nn.Conv2d(in_channels=256,out_channels=384,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            # layer 4
            nn.Conv2d(in_channels=384,out_channels=384,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            # layer 5
            nn.Conv2d(in_channels=384,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3,stride=2)
        )
        self.fc=nn.Sequential(
            # layer 6
            nn.Linear(in_features=6*6*256,out_features=4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            # layer 7
            nn.Linear(in_features=4096,out_features=4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            # layer 8
            nn.Linear(in_features=4096,out_features=num_classes)
        )
        def forward(self,img):
            feature=self.conv(img)
            # output=self.fc(feature.view(img.shape[0],-1))
            output=self.fc(feature.view(-1,6*6*256))
            return output

In [4]:
# print net
net=AlexNet()
print(net)

AlexNet(
  (conv): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=2)
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=9216, out_features=4096, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_feat

## 读取数据

读取数据的时候我们额外做了一步将图像高和宽扩大到AlexNet使用的图像高和宽224。

这个可以通过torchvision.transforms.Resize实例来实现。

也就是说，我们在ToTensor实例前使用Resize实例，然后使用Compose实例来将这两个变换串联以方便调用。

In [10]:
# 构建一个列表：将多个图像变换实例集合到一起

resize=227
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize(size=resize),
    transforms.ToTensor(),
  	# Normalize 这8个值是针对 CIFAR-10 这个数据集算出来的，对于其他数据集不适用
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

In [12]:
# CIFAR10 dataset
train_dataset = torchvision.datasets.CIFAR10(
    root='../datasets/', 
    train=True, 
    download=False, 
    transform=transform)

test_dataset= torchvision.datasets.CIFAR10(
    root='../datasets/', 
    train=False,
    download=False, 
    transform=transform)


100.0%Extracting ../datasets/cifar-10-python.tar.gz to ../datasets/
Files already downloaded and verified


In [ ]:
batch_size=128
train_iter=torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=4)
test_iter=torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=4)
